In [5]:
import warnings
warnings.filterwarnings("ignore")

# Prompt Templates

In [4]:
import faiss
from sentence_transformers import SentenceTransformer
from langchain import PromptTemplate

# Cargar el modelo de embeddings
embedding_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Textos de ejemplo para la base de datos
documents = [
    "Python es un lenguaje de programación interpretado, interactivo y orientado a objetos.",
    "La capital de Francia es París, conocida por su arquitectura, cultura y gastronomía.",
    "La inteligencia artificial se refiere a la simulación de procesos de inteligencia humana por parte de sistemas informáticos.",
    "El café es una bebida elaborada a partir de granos tostados de la planta de café."
]

# Convertir los documentos a vectores de embeddings
doc_vectors = embedding_model.encode(documents).astype('float32')

# Crear el índice de FAISS
d = doc_vectors.shape[1]  # Dimensionalidad de los vectores
index = faiss.IndexFlatL2(d)
index.add(doc_vectors)

# Función para buscar los documentos más similares usando FAISS
def search_documents(query, k=2):
    query_vector = embedding_model.encode([query]).astype('float32')
    distances, indices = index.search(query_vector, k)
    return [documents[idx] for idx in indices[0]]

from langchain.llms import CTransformers

# Cargar el modelo y el tokenizer de LLaMA
llama_model = "TheBloke/Llama-2-7B-Chat-GGML"
model = CTransformers(
            model = llama_model,
            model_type="llama",
            max_new_tokens = 512,
            temperature = 0.5
        )


# Crear una cadena de LLM utilizando LangChain
prompt_template = PromptTemplate(input_variables=["context", "question"], 
                                 template="Usa la siguiente información para responder la pregunta.\n\nContexto: {context}\n\nPregunta: {question}\n\nRespuesta:")

def generate_response(context, question):
    input_text = prompt_template.format(context=context, question=question)
    response = model.invoke(input_text)
    return response

# Función de generación aumentada por recuperación
def rag_generate(question):
    context = search_documents(question)
    context_str = " ".join(context)
    response = generate_response(context_str, question)
    return response

# Ejemplo de uso
question = "¿Qué es la inteligencia artificial?"
response = rag_generate(question)
print("Pregunta:", question)
print("Respuesta:", response)


c:\Users\vborb\Desktop\shareables\llama_faiss_lucy\llama_faiss\notebooks\faiss_rag\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Fetching 1 files: 100%|██████████| 1/1 [00:00<00:00, 691.22it/s]


Pregunta: ¿Qué es la inteligencia artificial?
Respuesta:  La inteligencia artificial (IA) es el campo de investigación que busca simular y mejorar la capacidad de los sistemas informáticos para resolver problemas complejos, como el reconocimiento de voz, la visión por computadora, el procesamiento de lenguaje natural, entre otros. La IA se basa en algoritmos y modelos que imitan los patrones y comportamientos humanos, permitiendo la interacción con sistemas de apoyo.


## Prompt

In [9]:
print(prompt_template.format(context="contexto", question="pregunta"))

Usa la siguiente información para responder la pregunta.

Contexto: contexto

Pregunta: pregunta

Respuesta:


# Similarity Search

In [1]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# Cargar el modelo de embeddings
embeddings_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Textos intuitivos de ejemplo
database_texts = [
    "Para hacer una tortilla de patatas, primero pela y corta las patatas en rodajas finas. Luego, fríelas en aceite caliente hasta que estén doradas.",
    "La receta del guacamole es simple: machaca los aguacates y mézclalos con tomate, cebolla, cilantro, y un poco de limón.",
    "El bosque estaba tranquilo, con árboles altos que se mecían suavemente con el viento. Un río serpenteaba a través del paisaje, reflejando el cielo azul.",
    "Desde la cima de la montaña, la vista era impresionante. Podías ver kilómetros de bosque, con el sol poniéndose en el horizonte."
]

query_texts = [
    "Para hacer una ensalada César, mezcla lechuga romana con trozos de pan tostado, queso parmesano, y aderezo César.",
    "La playa estaba desierta, con arena blanca y olas suaves que rompían en la orilla bajo el cielo despejado."
]

# Convertir los textos en embeddings
database_vectors = embeddings_model.encode(database_texts)
query_vectors = embeddings_model.encode(query_texts)

# Convertir los embeddings a tipo float32 (requerido por FAISS)
database_vectors = np.array(database_vectors).astype('float32')
query_vectors = np.array(query_vectors).astype('float32')

# Crear el índice de FAISS
d = database_vectors.shape[1]  # Dimensionalidad de los vectores
index = faiss.IndexFlatL2(d)  # Usar distancia euclidiana

# Agregar los vectores de la base de datos al índice
index.add(database_vectors)

# Realizar búsqueda de similitud
k = 2  # Número de vecinos más cercanos a recuperar
distances, indices = index.search(query_vectors, k)

# Mostrar resultados
for i, query in enumerate(query_texts):
    print(f"Consulta: {query}")
    print("Textos similares:")
    for j in range(k):
        print(f"  - {database_texts[indices[i, j]]} (Distancia: {distances[i, j]})")
    print()

c:\Users\vborb\Desktop\shareables\llama_faiss_lucy\llama_faiss\notebooks\.venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\vborb\Desktop\shareables\llama_faiss_lucy\llama_faiss\notebooks\.venv\Lib\site-packages\threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
c:\Users\vborb\Desktop\shareables\llama_faiss_lucy\llama_faiss\notebooks\.venv\Li

Consulta: Para hacer una ensalada César, mezcla lechuga romana con trozos de pan tostado, queso parmesano, y aderezo César.
Textos similares:
  - La receta del guacamole es simple: machaca los aguacates y mézclalos con tomate, cebolla, cilantro, y un poco de limón. (Distancia: 19.33415985107422)
  - El bosque estaba tranquilo, con árboles altos que se mecían suavemente con el viento. Un río serpenteaba a través del paisaje, reflejando el cielo azul. (Distancia: 21.67862319946289)

Consulta: La playa estaba desierta, con arena blanca y olas suaves que rompían en la orilla bajo el cielo despejado.
Textos similares:
  - El bosque estaba tranquilo, con árboles altos que se mecían suavemente con el viento. Un río serpenteaba a través del paisaje, reflejando el cielo azul. (Distancia: 18.767475128173828)
  - La receta del guacamole es simple: machaca los aguacates y mézclalos con tomate, cebolla, cilantro, y un poco de limón. (Distancia: 23.12240219116211)



## Embeddings

In [13]:
print("Longitud de database_vectors:", len(database_vectors))
print("Longitud de query_vectors:", len(query_vectors))
print("Longitud del primer elemento de database_vectors:", len(database_vectors[0]))
print("Longitud del primer elemento de query_vectors:", len(query_vectors[0]))

Longitud de database_vectors: 4
Longitud de query_vectors: 2
Longitud del primer elemento de database_vectors: 384
Longitud del primer elemento de query_vectors: 384


In [40]:
print(f'{query_texts[0][:50]}...:', f'{f'{query_vectors[0][:3]}'[:-1]}..]')
print(f'{database_texts[1][:50]}...:', f'{f'{database_vectors[1][:3]}'[:-1]}..]')

Para hacer una ensalada César, mezcla lechuga roma...: [-0.01954905  0.17426275  0.0006641 ..]
La receta del guacamole es simple: machaca los agu...: [-0.07286499  0.17364433 -0.14115788..]
